In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('map/map_wireframe.png')

In [7]:
# define the source and destination points
src_pts = np.array([[3330, -5300], [ 480, 1450], [-3370, -5300], [2680, 7400], [-2230, 7450]], dtype=np.float32)
dst_pts = np.array([[8530, 430], [6800, 4550], [4480, 430], [8180, 8150], [5200, 8150]], dtype=np.float32)

# compute the homography matrix
H, _ = cv2.findHomography(src_pts, dst_pts)
H

array([[ 5.97761956e-01,  3.80908051e-03,  6.51975427e+03],
       [-1.10863227e-03,  6.07125562e-01,  3.64844396e+03],
       [-1.12178794e-06,  9.23339614e-08,  1.00000000e+00]])

In [5]:
import pandas as pd
df = pd.read_csv('raw.csv')

In [6]:
df.loc[0]

tx           TX_F1_Fountain
x                475.289673
y               1482.590576
z                102.184547
rssi                    -58
timestamp     1679538902693
Name: 0, dtype: object

In [7]:
inx = df.loc[0].x
iny = df.loc[0].y

point = np.array([[inx, iny]], dtype=np.float32)
# apply the homography projection to the point
projected_point = cv2.perspectiveTransform(point.reshape(1, -1, 2), H)
point = projected_point[0][0]
point

array([6812.2114, 4549.839 ], dtype=float32)

In [4]:
def ab(df):
    return df.values

df = df.groupby(['timestamp', 'x', 'y', 'z'])['rssi'].apply(ab)

NameError: name 'df' is not defined

In [10]:
df = df.to_frame().reset_index()
df

,timestamp,x,y,z,rssi
0,1679538902693,475.289673,1482.590576,102.184547,"[-58, -70, -67, -66, -62, -58]"
1,1679538902759,476.011902,1483.588623,110.404449,"[-57, -72, -67, -67, -63, -58]"
2,1679538902794,486.526764,1481.530273,110.504608,"[-57, -69, -65, -65, -64, -59]"
3,1679538902828,496.901855,1474.441528,110.863770,"[-57, -71, -66, -67, -63, -60]"
4,1679538902860,506.555115,1461.993652,111.978073,"[-57, -70, -67, -66, -62, -60]"
...,...,...,...,...,...
6292,1679539160779,-693.878540,7103.696777,114.846436,"[-74, -73, -61, -65, -55, -58, -64]"
6293,1679539160804,-693.678711,7106.986816,114.070290,"[-73, -74, -62, -65, -54, -58, -64]"
6294,1679539160831,-693.429626,7110.733398,113.161743,"[-73, -75, -61, -65, -54, -58, -64]"
6295,1679539160858,-693.255554,7114.434570,112.097168,"[-73, -74, -62, -65, -54, -58, -64]"


In [8]:
img = cv2.imread('map/map_wireframe.png')
for index, row in df.iterrows():
    inx = int(row['x'])
    iny = int(row['y'])
    point = np.array([[inx, iny]], dtype=np.float32)
    projected_point = cv2.perspectiveTransform(point.reshape(1, -1, 2), H)
    point = projected_point[0][0]
    # Window name in which image is displayed
    window_name = 'Image'
    # Center coordinates
    center_coordinates = (int(point[0]), int(point[1]))
    # Radius of circle
    radius = 20
    # Blue color in BGR
    color = (255,215,0)
    # Line thickness of 2 px
    thickness = 20

    cv2.circle(img, center_coordinates, radius, color, thickness)
img = cv2.resize(img, (1920, 1280))
cv2.imshow(window_name, img)
cv2.waitKey(0)
cv2.destroyAllWindows()